# Physical Object: Lake

The `Lake` class models a lake or reservoir with a fixed surface area. It's a stateful object, meaning its state (primarily its volume) evolves over time based on inflows and outflows.

This model accounts for:
- **Inflow**: Water entering the lake from upstream sources.
- **Outflow**: Water being drawn from the lake by a downstream component (like a turbine or gate).
- **Evaporation**: Water lost from the surface, calculated from an evaporation rate.

### State Variables
- `volume` (float): The current volume of water in the lake (m^3).
- `water_level` (float): The current water level (m), calculated from volume and surface area.
- `outflow` (float): The outflow from the lake for the current step (m^3/s). This is not calculated by the lake itself, but is provided as an input to its `step` method.

### Parameters
- `surface_area` (float): The surface area of the lake (m^2).
- `max_volume` (float): The maximum storage capacity of the lake (m^3).
- `evaporation_rate_m_per_s` (float): The rate of evaporation in meters per second.

## Simulation Example

In the simulation below, we model a lake with a constant inflow and a variable outflow demand from a downstream component. We can observe how the lake's volume and water level are affected by the balance between inflow, outflow, and evaporation.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from swp.simulation_identification.physical_objects.lake import Lake
from swp.core.interfaces import State, Parameters

# 1. Define initial state and parameters
initial_volume = 40e6
surface_area = 2e6
initial_state = State(volume=initial_volume, water_level=initial_volume/surface_area, outflow=0)
parameters = Parameters(
    surface_area=surface_area,
    max_volume=50e6,
    evaporation_rate_m_per_s=2.31e-8 # Equivalent to 2mm/day
)

# 2. Create a Lake instance
lake = Lake(name="upper_lake", initial_state=initial_state, parameters=parameters)

# 3. Simulation settings
dt = 3600 # 1-hour time steps
simulation_duration = 86400 * 5 # 5 days
num_steps = int(simulation_duration / dt)
history = []
inflows = []
requested_outflows = []

# 4. Run the simulation loop
for t in range(num_steps):
    # Constant inflow
    inflow = 100.0
    inflows.append(inflow)
    lake.set_inflow(inflow)
    
    # Variable outflow demand (e.g. from a power station)
    # High demand during the day, low at night
    hour_of_day = (t * dt / 3600) % 24
    if 8 <= hour_of_day <= 20:
        requested_outflow = 150.0
    else:
        requested_outflow = 50.0
    requested_outflows.append(requested_outflow)
    
    action = {'outflow': requested_outflow}
    
    current_state = lake.step(action=action, dt=dt)
    history.append(current_state.copy())

print("Lake simulation complete.")

## Results and Visualization

The plots show the lake's water level responding to the daily cycle of outflow demand. During the day, when outflow exceeds inflow, the water level drops. At night, when inflow exceeds outflow, the level recovers. A slight downward trend is also visible due to constant evaporation loss.

In [ ]:
# Extract data from history
time_days = [i * dt / 86400 for i in range(num_steps)]
water_levels = [h['water_level'] for h in history]
volumes_Mm3 = [h['volume'] / 1e6 for h in history] # Convert to million cubic meters
actual_outflows = [h['outflow'] for h in history]

# Create a DataFrame
df = pd.DataFrame({
    'Time (days)': time_days,
    'Inflow (m^3/s)': inflows,
    'Requested Outflow (m^3/s)': requested_outflows,
    'Actual Outflow (m^3/s)': actual_outflows,
    'Water Level (m)': water_levels,
    'Volume (Mm^3)': volumes_Mm3
})

print(df.head())

# Plot the results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Flows
ax1.plot(df['Time (days)'], df['Inflow (m^3/s)'], label='Inflow', linestyle='--')
ax1.plot(df['Time (days)'], df['Requested Outflow (m^3/s)'], label='Requested Outflow', linestyle=':')
ax1.plot(df['Time (days)'], df['Actual Outflow (m^3/s)'], label='Actual Outflow')
ax1.set_ylabel('Flow Rate (m^3/s)')
ax1.set_title('Lake Simulation with Daily Demand Cycle')
ax1.grid(True)
ax1.legend()

# Water Level
ax2.plot(df['Time (days)'], df['Water Level (m)'], label='Water Level', color='purple')
ax2.set_xlabel('Time (days)')
ax2.set_ylabel('Water Level (m)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()